In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import requests
from config import weather_api_key
from datetime import datetime

In [3]:
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [4]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [5]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

616

In [6]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=bbfca4408d523e005bca6d75c518b0d0


In [7]:
# Create an endpoint URL for a city.
city_url = url + "&q=" + "Boston"
print(city_url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=bbfca4408d523e005bca6d75c518b0d0&q=Boston


In [8]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | albany
Processing Record 2 of Set 1 | hamilton
Processing Record 3 of Set 1 | punta arenas
Processing Record 4 of Set 1 | hasaki
Processing Record 5 of Set 1 | nikolskoye
Processing Record 6 of Set 1 | beringovskiy
Processing Record 7 of Set 1 | bredasdorp
Processing Record 8 of Set 1 | livingstone
Processing Record 9 of Set 1 | hobart
Processing Record 10 of Set 1 | chicama
Processing Record 11 of Set 1 | mataura
Processing Record 12 of Set 1 | vaitupu
City not found. Skipping...
Processing Record 13 of Set 1 | tuktoyaktuk
Processing Record 14 of Set 1 | yenagoa
Processing Record 15 of Set 1 | saskylakh
Processing Record 16 of Set 1 | castro
Processing Record 17 of Set 1 | busselton
Processing Record 18 of Set 1 | mar del plata
Processing Record 19 of Set 1 | atuona
Processing Record 20 of Set 1 | new norfolk
Processing Record 21 of Set 1 | puerto ayora
Processing Record 22 of Set 1 | baniachang

In [9]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Albany,42.60,-73.97,69.01,58,0,8.16,US,2020-08-26 03:46:10
1,Hamilton,39.18,-84.53,75.99,94,1,2.77,US,2020-08-26 03:42:04
2,Punta Arenas,-53.15,-70.92,28.40,86,70,3.36,CL,2020-08-26 03:43:44
3,Hasaki,35.73,140.83,91.00,70,75,8.05,JP,2020-08-26 03:46:10
4,Nikolskoye,59.70,30.79,57.20,87,75,4.47,RU,2020-08-26 03:44:00
5,Beringovskiy,63.05,179.32,53.11,76,83,15.68,RU,2020-08-26 03:46:11
6,Bredasdorp,-34.53,20.04,48.20,87,99,3.36,ZA,2020-08-26 03:45:56
7,Livingstone,-17.84,25.85,58.33,41,0,2.53,ZM,2020-08-26 03:46:11
8,Hobart,-42.88,147.33,55.99,54,75,14.99,AU,2020-08-26 03:46:11
9,Chicama,-7.84,-79.15,59.00,84,15,4.54,PE,2020-08-26 03:46:11


In [10]:
new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
city_data_df = city_data_df[new_column_order]
city_data_df.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Albany,US,2020-08-26 03:46:10,42.60,-73.97,69.01,58,0,8.16
1,Hamilton,US,2020-08-26 03:42:04,39.18,-84.53,75.99,94,1,2.77
2,Punta Arenas,CL,2020-08-26 03:43:44,-53.15,-70.92,28.40,86,70,3.36
3,Hasaki,JP,2020-08-26 03:46:10,35.73,140.83,91.00,70,75,8.05
4,Nikolskoye,RU,2020-08-26 03:44:00,59.70,30.79,57.20,87,75,4.47


In [11]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")

FileNotFoundError: [Errno 2] No such file or directory: 'weather_data/cities.csv'